In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import json

In [2]:
project_root = Path.cwd().parent
src_path = project_root / 'src'

if src_path not in sys.path:
    sys.path.append(str(src_path))

In [3]:
env_path = project_root / '.env'
load_dotenv(env_path)

True

In [4]:
GIGA_CREDENTIALS = os.getenv('GIGACHAT_CREDENTIALS')
print(f'Ключ загружен: {GIGA_CREDENTIALS is not None}')

Ключ загружен: True


In [5]:
# Генерация реалистичных тестовых данных для разных сценариев
sample_reviews = [
    # Негативные отзывы
    {"text": "Товар пришел с царапиной на экране. Доставка была быстрая, но качество подкачало.", "source": "ozon"},
    {"text": "Ждал доставку 2 недели! Никаких извинений и оповещений. Ужасный сервис.", "source": "wildberries"},
    {"text": "Поддержка игнорирует мой запрос уже пятый день. Проблема с возвратом.", "source": "сайт"},
    {"text": "Цена явно завышена. У конкурентов такой же товар на 30% дешевле.", "source": "yandex_market"},
    # Позитивные отзывы
    {"text": "Отличный продукт! Работает безупречно, дизайн шикарный. Рекомендую!", "source": "сайт"},
    {"text": "Доставили раньше срока, курьер вежливый. Упаковка целая. Супер!", "source": "ozon"},
    {"text": "Менеджер помог подобрать идеальную конфигурацию под мой бюджет. Очень благодарен!", "source": "телефон"},
    {"text": "Соотношение цена-качество просто отличное. Лучшая покупка за последнее время.",
     "source": "yandex_market"},
    # Нейтральные/смешанные отзывы
    {"text": "Товар неплохой, но и не шедевр. Доставка средняя, без восторгов.", "source": "wildberries"},
    {"text": "Функционал хороший, но интерфейс запутанный. Пришлось долго разбираться.", "source": "сайт"},
]

In [6]:
df_reviews = pd.DataFrame(sample_reviews)

In [9]:
df_reviews['id'] = range(1, len(df_reviews) + 1)

In [11]:
df_reviews = df_reviews.reindex(columns=['id', 'text', 'source'])

In [12]:
df_reviews.head()

,id,text,source
0,1,Товар пришел с царапиной на экране. Доставка б...,ozon
1,2,Ждал доставку 2 недели! Никаких извинений и оп...,wildberries
2,3,Поддержка игнорирует мой запрос уже пятый день...,сайт
3,4,Цена явно завышена. У конкурентов такой же тов...,yandex_market
4,5,"Отличный продукт! Работает безупречно, дизайн ...",сайт


In [13]:
raw_data_path = project_root / 'data' / 'raw'
raw_data_path.mkdir(parents=True, exist_ok=True)
file_path = raw_data_path / 'sample_reviews.csv'
df_reviews.to_csv(file_path, index=False, encoding='utf-8-sig')

In [14]:
from transformers import pipeline

D:\ProgramData\anaconda3\envs\insightagent\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


  0%|          | 0/6 [00:00<?, ?it/s]

D:\ProgramData\anaconda3\envs\insightagent\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [20]:
sentiment_analyzer = pipeline('sentiment-analysis',
                              model='blanchefort/rubert-base-cased-sentiment',
                              device=-1)

D:\ProgramData\anaconda3\envs\insightagent\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
def analyze_sentiment(texts):
    results = sentiment_analyzer(texts)
    
    sentiment_map = {'NEGATIVE': 'negative', 'NEUTRAL': 'neutral', 'POSITIVE': 'positive'}
    return [{'sentiment': sentiment_map[r['label']],
             'confidence': r['score']} for r in results]

In [22]:
texts = df_reviews['text'].tolist()
sentiment_results = analyze_sentiment(texts)

In [23]:
df_reviews['sentiment'] = [r['sentiment'] for r in sentiment_results]
df_reviews['sentiment_confidence'] = [r['confidence'] for r in sentiment_results]

In [25]:
df_reviews.head(10)

,id,text,source,sentiment,sentiment_confidence
0,1,Товар пришел с царапиной на экране. Доставка б...,ozon,neutral,0.679798
1,2,Ждал доставку 2 недели! Никаких извинений и оп...,wildberries,negative,0.751454
2,3,Поддержка игнорирует мой запрос уже пятый день...,сайт,negative,0.751476
3,4,Цена явно завышена. У конкурентов такой же тов...,yandex_market,neutral,0.754552
4,5,"Отличный продукт! Работает безупречно, дизайн ...",сайт,positive,0.980789
5,6,"Доставили раньше срока, курьер вежливый. Упако...",ozon,positive,0.980823
6,7,Менеджер помог подобрать идеальную конфигураци...,телефон,positive,0.975352
7,8,Соотношение цена-качество просто отличное. Луч...,yandex_market,positive,0.980598
8,9,"Товар неплохой, но и не шедевр. Доставка средн...",wildberries,neutral,0.689301
9,10,"Функционал хороший, но интерфейс запутанный. П...",сайт,neutral,0.817163


In [27]:
from langchain_gigachat import GigaChat
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import time

In [28]:
llm = GigaChat(
    credentials=GIGA_CREDENTIALS,
    model='GigaChat-2',
    verify_ssl_certs=False,
    temperature=0,
    timeout=60
)

classification_template = '''
Ты — опытный аналитик обратной связи. Определи основную тему отзыва клиента.
Тема должна быть одной из следующих категорий: [Качество товара, Доставка, Служба поддержки, Цена, Общее впечатление].

Отзыв: {review_text}

Верни ТОЛЬКО название категории, без кавычек, точек и дополнительного текста.
'''

prompt = PromptTemplate(template=classification_template, input_variables=['review_text'])
classification_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)
def classify_review(text):
    try:
        result = classification_chain.run(review_text = text)
        
        return result.strip()
    except Exception as e:
        print(f'Ошибка классификации: {e}')
        return 'Ошибка'
    
df_reviews['predicted_theme'] = df_reviews['text'].apply(classify_review)

C:\Users\User\AppData\Local\Temp\ipykernel_15388\429806637.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  classification_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_15388\429806637.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = classification_chain.run(review_text = text)


In [29]:
df_reviews[['text', 'sentiment', 'predicted_theme']]

,text,sentiment,predicted_theme
0,Товар пришел с царапиной на экране. Доставка б...,neutral,Качество товара
1,Ждал доставку 2 недели! Никаких извинений и оп...,negative,Доставка
2,Поддержка игнорирует мой запрос уже пятый день...,negative,Служба поддержки
3,Цена явно завышена. У конкурентов такой же тов...,neutral,Цена
4,"Отличный продукт! Работает безупречно, дизайн ...",positive,Качество товара
5,"Доставили раньше срока, курьер вежливый. Упако...",positive,Доставка
6,Менеджер помог подобрать идеальную конфигураци...,positive,Служба поддержки
7,Соотношение цена-качество просто отличное. Луч...,positive,Цена
8,"Товар неплохой, но и не шедевр. Доставка средн...",neutral,Доставка
9,"Функционал хороший, но интерфейс запутанный. П...",neutral,Интерфейс


In [30]:
insight_template = '''
На основе отзыва клиента сгенерируй структурированный анализ

Отзыв: {review_text}
Тональность: {sentiment}
Тема: {theme}

Проанализируй и верни ответ в формату JSON:
{{
    "key_problem_or_strength": "Основная проблема или преимущество (одно предложение)",
    "root_cause": "Возможная причина проблемы (если есть)",
    "actionable_recommendation": "Конкретная рекомендация для бизнеса",
    "priority": "Приоритет (high/medium/low)"
}}
Только JSON, никакого другого текста. Старайся быть довольно кратким.
'''

insight_prompt = PromptTemplate(
    template=insight_template,
    input_variables=['review_text', 'sentiment', 'theme']
)
insight_chain = LLMChain(llm=llm, prompt=insight_prompt, verbose=False)

def extract_insights(row):
    try:
        result = insight_chain.run(review_text=row['text'],
                                   sentiment=row['sentiment'],
                                   theme=row['predicted_theme'])
        
        parsed = json.loads(result.strip())
        return parsed
    except json.JSONDecodeError as e:
        print(f"Ошибка парсинка JSON для отзыва '{row['text'][:30]}'...: {e}")
        return {'error': 'JSON decode failed', 'raw_response': result}
    except Exception as e:
        print(f'Ошибка для отзыва {row["id"]}: {e}')
        return {'error': str(e)}

In [31]:
insights = df_reviews.apply(extract_insights, axis=1)

insights_df = pd.json_normalize(insights.tolist())
reviews_insights = pd.concat([df_reviews.reset_index(drop=True), insights_df], axis=1)

Ошибка парсинка JSON для отзыва 'Поддержка игнорирует мой запро'...: Expecting value: line 1 column 1 (char 0)


In [32]:
reviews_insights

,id,text,source,sentiment,sentiment_confidence,predicted_theme,key_problem_or_strength,root_cause,actionable_recommendation,priority,error,raw_response
0,1,Товар пришел с царапиной на экране. Доставка б...,ozon,neutral,0.679798,Качество товара,Качество товара подкачало из-за царапины на эк...,"Возможно, товар транспортировался небрежно или...",Усилить упаковку товаров перед отправкой клиен...,high,NaN,NaN
1,2,Ждал доставку 2 недели! Никаких извинений и оп...,wildberries,negative,0.751454,Доставка,Длительная доставка без уведомлений и извинений.,,Улучшить систему уведомлений клиентов о статус...,high,NaN,NaN
2,3,Поддержка игнорирует мой запрос уже пятый день...,сайт,negative,0.751476,Служба поддержки,NaN,NaN,NaN,NaN,JSON decode failed,"```json\n{\n ""key_problem_or_strength"": ""Подд..."
3,4,Цена явно завышена. У конкурентов такой же тов...,yandex_market,neutral,0.754552,Цена,Завышенная цена товара,,Провести анализ ценовых предложений конкуренто...,high,NaN,NaN
4,5,"Отличный продукт! Работает безупречно, дизайн ...",сайт,positive,0.980789,Качество товара,Продукт высокого качества с отличным дизайном ...,,Улучшить маркетинговые материалы для акцентиро...,high,NaN,NaN
5,6,"Доставили раньше срока, курьер вежливый. Упако...",ozon,positive,0.980823,Доставка,Доставка выполнена раньше срока и упаковка сох...,,Улучшить контроль качества упаковки и сроков д...,high,NaN,NaN
6,7,Менеджер помог подобрать идеальную конфигураци...,телефон,positive,0.975352,Служба поддержки,Идеально подобранная конфигурация под бюджет к...,,Обеспечить высокий уровень персонализированног...,high,NaN,NaN
7,8,Соотношение цена-качество просто отличное. Луч...,yandex_market,positive,0.980598,Цена,Высокая ценность покупки благодаря отличному с...,,Поддерживать конкурентоспособные цены и качест...,high,NaN,NaN
8,9,"Товар неплохой, но и не шедевр. Доставка средн...",wildberries,neutral,0.689301,Доставка,"Доставка товара оказалась средней, без особых ...",,"Улучшить качество доставки и сервиса, повысить...",medium,NaN,NaN
9,10,"Функционал хороший, но интерфейс запутанный. П...",сайт,neutral,0.817163,Интерфейс,Интерфейс неудобный и сложный для понимания.,,Провести юзабилити-тестирование и упростить ди...,high,NaN,NaN
